In [1]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # Linear algebra library
import pandas as pd

In [2]:
! pwd
expr_df = pd.read_csv("../metadata/length_and_depth.csv")
expr_df = expr_df.drop("nvar", axis=1)
expr_df = expr_df.rename(columns={
    "length" : "expr_length",
    "depth" : "expr_depth"
})
expr_df

/Users/felixgao/Desktop/Year3/STA378/OptimizationParameterTuning/Julia Notebook/src


,problem,objtype,variable_nvar,expr_length,expr_depth
0,NZF1,least_squares,True,250,8
1,arglina,least_squares,True,10404,9
2,arglinb,least_squares,True,40200,8
3,arglinc,other,True,39204,7
4,argtrig,other,True,497,7
...,...,...,...,...,...
100,tridia,other,True,398,6
101,vardim,other,True,798,6
102,vibrbeam,least_squares,False,481,13
103,watson,least_squares,False,3900,12


In [3]:
complete_df = pd.read_csv("../results/complete_dataset_as_of_nov6.csv")
complete_df = complete_df[complete_df["is_init_run"] == 0]
complete_df = complete_df.rename(columns={"name" : "problem"})
df = complete_df.merge(
    expr_df,
    on = 'problem',
    how = "inner"
)
df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,neval_grad,init_eval_grad_time,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth
0,first_order,NZF1,LBFGSSolver,1,91,1.409402,0.125416,287,287,328,...,315,0.000031,0.001440,5,False,False,least_squares,True,250,8
1,first_order,NZF1,LBFGSSolver,2,91,1.407668,0.095296,199,199,223,...,216,0.000019,0.001440,5,False,False,least_squares,True,250,8
2,first_order,NZF1,LBFGSSolver,3,91,1.407882,0.103456,213,213,234,...,228,0.000016,0.001440,5,False,False,least_squares,True,250,8
3,first_order,NZF1,LBFGSSolver,4,91,1.407476,0.098352,191,191,209,...,203,0.000017,0.001440,5,False,False,least_squares,True,250,8
4,first_order,NZF1,LBFGSSolver,5,91,1.407906,0.108416,209,209,231,...,225,0.000017,0.001440,5,False,False,least_squares,True,250,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18821,first_order,woods,LBFGSSolver,96,1000,1.482090,3.197624,64,64,84,...,75,0.001356,0.008664,5,False,True,other,True,425,7
18822,first_order,woods,LBFGSSolver,97,1000,1.481800,3.229800,64,64,84,...,75,0.001373,0.008664,5,False,True,other,True,425,7
18823,first_order,woods,LBFGSSolver,98,1000,1.482000,3.261976,64,64,84,...,75,0.001367,0.008664,5,False,True,other,True,425,7
18824,first_order,woods,LBFGSSolver,99,1000,1.481908,3.294152,64,64,84,...,75,0.001358,0.008664,5,False,True,other,True,425,7


In [4]:
group_cols = ["problem", "nvar"]

d = df[df["time"].notna()].copy()
d = d.sort_values(group_cols + ["time", "mem"], ascending=True)

best_idx = d.groupby(group_cols)["time"].idxmin()
best_mem_df = d.loc[best_idx].reset_index(drop=True)
best_mem_df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,neval_grad,init_eval_grad_time,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth
0,first_order,NZF1,LBFGSSolver,64,91,1.405412,0.241360,59,59,70,...,64,0.000017,0.001440,5,False,False,least_squares,True,250,8
1,first_order,NZF1,LBFGSSolver,72,988,3.127719,2.406504,81,81,94,...,86,0.001715,0.008600,5,False,True,least_squares,True,250,8
2,first_order,arglina,LBFGSSolver,4,100,0.004524,0.029440,1,1,3,...,3,0.000059,0.001520,5,False,False,least_squares,True,10404,9
3,first_order,arglina,LBFGSSolver,5,1000,1.400665,0.240920,1,1,3,...,3,0.005946,0.008664,5,False,True,least_squares,True,10404,9
4,first_order,arglinb,LBFGSSolver,1,100,1.422291,0.020488,2,2,29,...,5,0.000051,0.001520,5,False,False,least_squares,True,40200,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,first_order,vardim,LBFGSSolver,21,1000,28.733923,1.509576,1165,1165,6897,...,6873,0.004314,0.008664,5,False,True,other,True,798,6
180,first_order,vibrbeam,LBFGSSolver,11,8,0.578265,0.054456,103,103,179,...,126,0.000006,0.000720,5,False,False,least_squares,False,481,13
181,unbounded,watson,LBFGSSolver,51,31,1.019050,32.288992,1,1,8,...,8,0.001956,4.349024,42554,False,False,least_squares,False,3900,12
182,first_order,woods,LBFGSSolver,8,100,1.288446,0.060864,45,45,69,...,60,0.000016,0.001520,5,False,False,other,True,425,7


In [5]:
best_mem_map = d.loc[best_idx].set_index(group_cols)["mem"]

df = df.copy()
df["best_problem_mem"] = df.set_index(group_cols).index.map(best_mem_map)
df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,init_eval_grad_time,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth,best_problem_mem
0,first_order,NZF1,LBFGSSolver,1,91,1.409402,0.125416,287,287,328,...,0.000031,0.001440,5,False,False,least_squares,True,250,8,64
1,first_order,NZF1,LBFGSSolver,2,91,1.407668,0.095296,199,199,223,...,0.000019,0.001440,5,False,False,least_squares,True,250,8,64
2,first_order,NZF1,LBFGSSolver,3,91,1.407882,0.103456,213,213,234,...,0.000016,0.001440,5,False,False,least_squares,True,250,8,64
3,first_order,NZF1,LBFGSSolver,4,91,1.407476,0.098352,191,191,209,...,0.000017,0.001440,5,False,False,least_squares,True,250,8,64
4,first_order,NZF1,LBFGSSolver,5,91,1.407906,0.108416,209,209,231,...,0.000017,0.001440,5,False,False,least_squares,True,250,8,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18821,first_order,woods,LBFGSSolver,96,1000,1.482090,3.197624,64,64,84,...,0.001356,0.008664,5,False,True,other,True,425,7,8
18822,first_order,woods,LBFGSSolver,97,1000,1.481800,3.229800,64,64,84,...,0.001373,0.008664,5,False,True,other,True,425,7,8
18823,first_order,woods,LBFGSSolver,98,1000,1.482000,3.261976,64,64,84,...,0.001367,0.008664,5,False,True,other,True,425,7,8
18824,first_order,woods,LBFGSSolver,99,1000,1.481908,3.294152,64,64,84,...,0.001358,0.008664,5,False,True,other,True,425,7,8


In [6]:
# unique instances: one row per (problem, nvar)
instances = df[["problem", "nvar"]].drop_duplicates()

# shuffle instances
instances = instances.sample(frac=1, random_state=66).reset_index(drop=True)

n = len(instances)
n_train = int(0.7 * n)
n_valid = int(0.15 * n)

train_inst = instances.iloc[:n_train]
valid_inst = instances.iloc[n_train:n_train + n_valid]
test_inst  = instances.iloc[n_train + n_valid:]

# assign rows to splits by (problem, nvar)
train_df = df.merge(train_inst, on=["problem", "nvar"], how="inner").reset_index(drop=True)
valid_df = df.merge(valid_inst, on=["problem", "nvar"], how="inner").reset_index(drop=True)
test_df  = df.merge(test_inst,  on=["problem", "nvar"], how="inner").reset_index(drop=True)
train_df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,init_eval_grad_time,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth,best_problem_mem
0,first_order,NZF1,LBFGSSolver,1,91,1.409402,0.125416,287,287,328,...,0.000031,0.001440,5,False,False,least_squares,True,250,8,64
1,first_order,NZF1,LBFGSSolver,2,91,1.407668,0.095296,199,199,223,...,0.000019,0.001440,5,False,False,least_squares,True,250,8,64
2,first_order,NZF1,LBFGSSolver,3,91,1.407882,0.103456,213,213,234,...,0.000016,0.001440,5,False,False,least_squares,True,250,8,64
3,first_order,NZF1,LBFGSSolver,4,91,1.407476,0.098352,191,191,209,...,0.000017,0.001440,5,False,False,least_squares,True,250,8,64
4,first_order,NZF1,LBFGSSolver,5,91,1.407906,0.108416,209,209,231,...,0.000017,0.001440,5,False,False,least_squares,True,250,8,64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13121,first_order,woods,LBFGSSolver,96,1000,1.482090,3.197624,64,64,84,...,0.001356,0.008664,5,False,True,other,True,425,7,8
13122,first_order,woods,LBFGSSolver,97,1000,1.481800,3.229800,64,64,84,...,0.001373,0.008664,5,False,True,other,True,425,7,8
13123,first_order,woods,LBFGSSolver,98,1000,1.482000,3.261976,64,64,84,...,0.001367,0.008664,5,False,True,other,True,425,7,8
13124,first_order,woods,LBFGSSolver,99,1000,1.481908,3.294152,64,64,84,...,0.001358,0.008664,5,False,True,other,True,425,7,8


In [7]:
from sklearn.ensemble import RandomForestRegressor
feature_cols = ["nvar", 
                "expr_length", 
                "expr_depth", 
                "init_eval_obj_time", 
                "init_eval_grad_time",
                "mem",
                ]
target_col = "time"
X_train = train_df[feature_cols].to_numpy(dtype=float)
X_valid = valid_df[feature_cols].to_numpy(dtype=float)
X_test  = test_df[feature_cols].to_numpy(dtype=float)
t_test_full  = np.log(test_df[target_col].to_numpy(dtype=float))

t_train = np.log(train_df[target_col].to_numpy(dtype=float))
t_valid = np.log(valid_df[target_col].to_numpy(dtype=float))
t_test  = np.log(test_df[target_col].to_numpy(dtype=float))

In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

best_gb = None
best_score = np.inf
best_params = None

for n_estimators in range(100, 500, 10):
    gb = GradientBoostingRegressor(
        n_estimators=n_estimators
    )
    gb.fit(X_train, t_train)

    pred_valid_log = gb.predict(X_valid)
    mse_valid = mean_squared_error(t_valid, pred_valid_log)

    if mse_valid < best_score:
        best_score = mse_valid
        best_gb = gb
        best_params = n_estimators
        print(
            "New best:",
            "n =", n_estimators,
            "log1p MSE =", mse_valid,
        )
    else:
        print(
            "n =", n_estimators,
            "log1p MSE =", mse_valid,
        )

print("Best validation MSE (Gradient Boosting):", best_score)
print("Best n_estimator::", best_params)


New best: n = 100 log1p MSE = 1.6429636624151922
New best: n = 110 log1p MSE = 1.6281924686787221
n = 120 log1p MSE = 1.6520963572044691
n = 130 log1p MSE = 1.6611356829648178
n = 140 log1p MSE = 1.6666202941708406
n = 150 log1p MSE = 1.674614978447088
n = 160 log1p MSE = 1.7097361323683038
n = 170 log1p MSE = 1.7303323166936162
n = 180 log1p MSE = 1.7504062795597046
n = 190 log1p MSE = 1.743320355117843
n = 200 log1p MSE = 1.7499521905499407
n = 210 log1p MSE = 1.7521365477034923
n = 220 log1p MSE = 1.741742633575352
n = 230 log1p MSE = 1.7441363042294793
n = 240 log1p MSE = 1.7516557639932206
n = 250 log1p MSE = 1.7814120486608025
n = 260 log1p MSE = 1.7895375397188127
n = 270 log1p MSE = 1.8001091531276785
n = 280 log1p MSE = 1.8415113830311232
n = 290 log1p MSE = 1.8347951177988888
n = 300 log1p MSE = 1.998170194537482
n = 310 log1p MSE = 1.9917400911719323
n = 320 log1p MSE = 1.9968517936959607
n = 330 log1p MSE = 2.083037860670836
n = 340 log1p MSE = 2.0749252367246473
n = 350 lo

In [33]:
best_n_estimators = best_params

X_train_full = np.vstack([X_train, X_valid])
t_train_full = np.concatenate([t_train, t_valid])

final_gb = GradientBoostingRegressor(
    n_estimators=best_n_estimators
)
final_gb.fit(X_train_full, t_train_full)

,loss,'squared_error'
,learning_rate,0.1
,n_estimators,110
,subsample,1.0
,criterion,'friedman_mse'
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_depth,3
,min_impurity_decrease,0.0
,init,None


In [45]:
feature_cols_no_mem = [c for c in feature_cols if c != "mem"]
X_test  = test_df[feature_cols_no_mem]
group_cols = ["problem", "nvar"]
problem_feature_cols = [c for c in feature_cols if c != "mem"]
X_test = (
    test_df.sort_values(group_cols)
           .drop_duplicates(subset=group_cols, keep="first")[problem_feature_cols]
           .reset_index(drop=True)
)
X_test

def choose_best_mem(model, x_problem, mem_candidates):
    preds = []
    for mem in mem_candidates:
        x = np.concatenate([x_problem, [mem]])
        preds.append((mem, model.predict(x.reshape(1, -1))[0]))
    preds[1] = np.exp(preds[1])
    return min(preds, key=lambda x: x[1])
X_test

,nvar,expr_length,expr_depth,init_eval_obj_time,init_eval_grad_time
0,100,40200,8,0.000003,0.000051
1,100,1153,7,0.000003,0.000053
2,100,800,10,0.000003,0.000051
3,100,642,7,0.000003,0.000038
4,961,974,6,0.000015,0.004992
5,1000,588,7,0.000021,0.003633
6,100,3335,8,0.000005,0.000057
7,99,887,6,0.000007,0.000110
8,99,495,4,0.000017,0.000212
9,99,887,6,0.000018,0.000234


In [46]:
best_n_estimators = best_params

X_train_full = np.vstack([X_train, X_valid])
t_train_full = np.concatenate([t_train, t_valid])

final_rf = GradientBoostingRegressor(
    n_estimators=best_n_estimators
)
X_train_full

array([[9.100000e+01, 2.500000e+02, 8.000000e+00, 1.375600e-05,
        3.141000e-05, 1.000000e+00],
       [9.100000e+01, 2.500000e+02, 8.000000e+00, 1.964000e-06,
        1.938700e-05, 2.000000e+00],
       [9.100000e+01, 2.500000e+02, 8.000000e+00, 1.553000e-06,
        1.647200e-05, 3.000000e+00],
       ...,
       [1.000000e+03, 7.980000e+02, 6.000000e+00, 5.190000e-06,
        4.235358e-03, 9.800000e+01],
       [1.000000e+03, 7.980000e+02, 6.000000e+00, 1.413700e-05,
        4.209348e-03, 9.900000e+01],
       [1.000000e+03, 7.980000e+02, 6.000000e+00, 1.277500e-05,
        4.322154e-03, 1.000000e+02]], shape=(15926, 6))

In [47]:
group_cols = ["problem", "nvar"]
problem_cols = ["problem", "nvar", "mem", "best_problem_mem", "time"]

problems = (
    test_df.loc[test_df["mem"] == test_df["best_problem_mem"], problem_cols]
           .drop_duplicates(subset=group_cols, keep="first")
           .reset_index(drop=True)
           .rename(columns={"time": "best_time"})
)
problems

,problem,nvar,mem,best_problem_mem,best_time
0,arglinb,100,1,1,1.422291
1,bdqrtic,100,28,28,1.401975
2,broyden3d,100,10,10,1.377813
3,chainwoo,100,3,3,1.372517
4,curly,100,96,96,1.417469
5,dixmaanh,99,19,19,0.015107
6,dixmaanm,99,97,97,1.208349
7,dixmaano,99,92,92,0.053089
8,edensch,100,7,7,1.243068
9,eg2,100,3,3,1.249519


In [66]:
mem_candidates = np.arange(1, 101)
preds = []

for i, x_problem in enumerate(X_test.itertuples(index=False, name=None)):
    pred = choose_best_mem(final_gb, x_problem, mem_candidates)
    preds.append(pred)

    prob_name = problems.iloc[i]["problem"]
    nvar = problems.iloc[i]["nvar"]
    actual_best_mem = problems.iloc[i]["best_problem_mem"]

    print(
        f"Best predicted mem for {prob_name}({nvar}) is mem={pred[0]} "
        f"and the actual best mem is {actual_best_mem} with runtime={pred[1]}"
    )
preds

Best predicted mem for arglinb(100) is mem=6 and the actual best mem is 1 with runtime=-0.09221145153167713
Best predicted mem for bdqrtic(100) is mem=6 and the actual best mem is 28 with runtime=0.06964894141662771
Best predicted mem for broyden3d(100) is mem=6 and the actual best mem is 10 with runtime=0.11307980629470965
Best predicted mem for chainwoo(100) is mem=6 and the actual best mem is 3 with runtime=0.3894586085783956
Best predicted mem for curly(100) is mem=4 and the actual best mem is 96 with runtime=0.8475697310156116
Best predicted mem for dixmaanh(99) is mem=6 and the actual best mem is 19 with runtime=2.361886571408186
Best predicted mem for dixmaanm(99) is mem=6 and the actual best mem is 97 with runtime=0.5534216281597577
Best predicted mem for dixmaano(99) is mem=3 and the actual best mem is 92 with runtime=-1.2504439033749615
Best predicted mem for edensch(100) is mem=6 and the actual best mem is 7 with runtime=-0.06284341663099444
Best predicted mem for eg2(100) i

[(np.int64(6), np.float64(-0.09221145153167713)),
 (np.int64(6), np.float64(0.06964894141662771)),
 (np.int64(6), np.float64(0.11307980629470965)),
 (np.int64(6), np.float64(0.3894586085783956)),
 (np.int64(4), np.float64(0.8475697310156116)),
 (np.int64(6), np.float64(2.361886571408186)),
 (np.int64(6), np.float64(0.5534216281597577)),
 (np.int64(3), np.float64(-1.2504439033749615)),
 (np.int64(6), np.float64(-0.06284341663099444)),
 (np.int64(6), np.float64(-1.7844811193366352)),
 (np.int64(1), np.float64(0.5488575549029415)),
 (np.int64(6), np.float64(0.2506916089302658)),
 (np.int64(6), np.float64(0.45559430519225413)),
 (np.int64(6), np.float64(1.6320030584848615)),
 (np.int64(6), np.float64(0.15225277678528112)),
 (np.int64(3), np.float64(3.984532003435175)),
 (np.int64(6), np.float64(-0.33764981374581365)),
 (np.int64(6), np.float64(0.36907129092252744)),
 (np.int64(6), np.float64(-1.02310200765751)),
 (np.int64(6), np.float64(4.052605985758634)),
 (np.int64(6), np.float64(3.010

In [67]:
problems["predicted_mem"] = [pred[0] for pred in preds]
problems

,problem,nvar,best_problem_mem,best_time,predicted_mem,predicted_mem_time
0,arglinb,100,1,1.422291,6,1.422297
1,bdqrtic,100,28,1.401975,6,1.404013
2,broyden3d,100,10,1.377813,6,1.378097
3,chainwoo,100,3,1.372517,6,1.396547
4,curly,100,96,1.417469,4,1.561085
5,dixmaanh,99,19,0.015107,6,0.015485
6,dixmaanm,99,97,1.208349,6,1.280845
7,dixmaano,99,92,0.053089,3,0.108117
8,edensch,100,7,1.243068,6,1.243165
9,eg2,100,3,1.249519,6,1.249520


In [68]:
feature_cols = ["nvar", 
                "expr_length", 
                "expr_depth", 
                "init_eval_obj_time", 
                "init_eval_grad_time",
                "mem",
                ]
feature_cols

['nvar',
 'expr_length',
 'expr_depth',
 'init_eval_obj_time',
 'init_eval_grad_time',
 'mem']

In [69]:
X_test_full = test_df[feature_cols].to_numpy(dtype=float)
y_pred_test = final_gb.predict(X_test_full)

test_mse = mean_squared_error(t_test, y_pred_test)
test_mae = mean_absolute_error(t_test, y_pred_test)
test_r2  = r2_score(t_test, y_pred_test)

print("Gradient Boosting MSE:", test_mse)
print("Gradient Boosting test MAE:", test_mae)
print("Gradient Boosting test R^2:", test_r2)

Gradient Boosting MSE: 1.4540149319839504
Gradient Boosting test MAE: 0.7400425941804472
Gradient Boosting test R^2: 0.46606061377546926


In [70]:
preds
actual_best_mem = problems["best_problem_mem"]
actual_best_mem

0      1
1     28
2     10
3      3
4     96
5     19
6     97
7     92
8      7
9      3
10     1
11    95
12    38
13    16
14    68
15    91
16    15
17    20
18    70
19    12
20     7
21    11
22     3
23    92
24    25
25    34
26    98
27    72
28     5
Name: best_problem_mem, dtype: int64

In [71]:
predicted_mem = np.array([pred[0] for pred in preds], dtype=int)
actual_best_mem = np.array(actual_best_mem, dtype=int)
acc = np.mean(predicted_mem == actual_best_mem)
print(f"Mem exact match accuracy: {acc:.2f}")

Mem exact match accuracy: 0.03


In [72]:
predicted_time = np.exp(np.array([pred[1] for pred in preds], dtype=float))
best_time = problems["best_time"]
ratios = predicted_time / best_time
within_5  = np.mean(ratios <= 1.05)
within_10 = np.mean(ratios <= 1.10)
print(f"Median time ratio: {np.median(ratios):.2f}")
print(f"Within 5% of optimal: {within_5:.2f}")
print(f"Within 10% of optimal: {within_10:.2f}")

Median time ratio: 1.37
Within 5% of optimal: 0.34
Within 10% of optimal: 0.41


In [77]:
# Filter for memory 5 first
mem_5_df = df[df["mem"] == 5]

# Merge with problems to filter for the exact problem+nvar pairs
baseline_df = pd.merge(mem_5_df, problems[["problem", "nvar"]], on=["problem", "nvar"], how="inner")
baseline_df
problems["baseline_time"] = baseline_df["time"]
problems

,problem,nvar,best_problem_mem,best_time,predicted_mem,predicted_mem_time,baseline_time
0,arglinb,100,1,1.422291,6,1.422297,1.422292
1,bdqrtic,100,28,1.401975,6,1.404013,1.405004
2,broyden3d,100,10,1.377813,6,1.378097,1.378392
3,chainwoo,100,3,1.372517,6,1.396547,1.389599
4,curly,100,96,1.417469,4,1.561085,1.641393
5,dixmaanh,99,19,0.015107,6,0.015485,0.015825
6,dixmaanm,99,97,1.208349,6,1.280845,1.272110
7,dixmaano,99,92,0.053089,3,0.108117,0.082949
8,edensch,100,7,1.243068,6,1.243165,1.243179
9,eg2,100,3,1.249519,6,1.249520,1.249519


In [78]:
# 1. Clean up 'problems' to remove any old merge artifacts
cols_to_drop = [c for c in ['mem', 'mem_x', 'mem_y', 'time_x', 'time_y', 'predicted_mem_time'] if c in problems.columns]
problems = problems.drop(columns=cols_to_drop)

# 2. Prepare the lookup slice from the original data
# We use drop_duplicates to ensure a 1-to-1 mapping
lookup = df[["problem", "nvar", "mem", "time"]].drop_duplicates(subset=["problem", "nvar", "mem"])
lookup

# 3. Perform the merge
problems = pd.merge(
    problems, 
    lookup, 
    left_on=["problem", "nvar", "predicted_mem"], 
    right_on=["problem", "nvar", "mem"], 
    how="left"
)

# 4. Final Cleanup: Rename 'time' and drop the redundant 'mem'
problems = problems.rename(columns={"time": "predicted_mem_time"}).drop(columns=["mem"])
problems

,problem,nvar,best_problem_mem,best_time,predicted_mem,baseline_time,predicted_mem_time
0,arglinb,100,1,1.422291,6,1.422292,1.422297
1,bdqrtic,100,28,1.401975,6,1.405004,1.404013
2,broyden3d,100,10,1.377813,6,1.378392,1.378097
3,chainwoo,100,3,1.372517,6,1.389599,1.396547
4,curly,100,96,1.417469,4,1.641393,1.561085
5,dixmaanh,99,19,0.015107,6,0.015825,0.015485
6,dixmaanm,99,97,1.208349,6,1.272110,1.280845
7,dixmaano,99,92,0.053089,3,0.082949,0.108117
8,edensch,100,7,1.243068,6,1.243179,1.243165
9,eg2,100,3,1.249519,6,1.249519,1.249520


In [79]:
# 1. Calculate Relative Gaps (as percentages)
problems['gap_default'] = (problems['baseline_time'] - problems['best_time']) / problems['best_time']
problems['gap_model'] = (problems['predicted_mem_time'] - problems['best_time']) / problems['best_time']

# 2. Square the gaps to penalize "bad" choices more heavily
# This is like calculating a 'Variance of Suboptimality'
baseline_mean = np.mean(problems['gap_default'])
predicted_mean = np.mean(problems['gap_model'])

# 3. Improvement in "Reliability"
reliability_improvement = (1 - (predicted_mean / baseline_mean)) * 100
print(f"Our Predicted mem is {reliability_improvement:.2f}% better than the default mem=5")

Our Predicted mem is 5.04% better than the default mem=5


In [ ]:
gb_tree0 = final_gb.estimators_[0, 0]
from sklearn import tree

plt.figure(figsize=(18, 10))
tree.plot_tree(
    gb_tree0,
    feature_names=feature_cols,
    filled=True,
    rounded=True,
    max_depth=3,
    fontsize=8,
    precision=6,
)
plt.title("Gradient boosting: tree 0 (top 3 levels)")
plt.tight_layout()
plt.show()


In [ ]:
from sklearn.tree import export_graphviz
import os

dot_data = export_graphviz(
    gb_tree0,
    max_depth=6,
    out_file=None,               # return string instead of writing directly to file
    feature_names=feature_cols,
    filled=True,
    rounded=True,
    precision=6                  # <-- more decimal places
)


In [ ]:
from graphviz import Source

os.makedirs("../tree_plots", exist_ok=True)

tree_graph = Source(dot_data)
pdf_path = tree_graph.render(
    filename="gradient_boosting",    # base name
    directory="../tree_plots",
    format="pdf",
    cleanup=True                     # delete intermediate .dot
)
print("PDF written to:", pdf_path)
tree_graph

In [ ]:
gb_importances = final_gb.feature_importances_
indices_gb = np.argsort(gb_importances)[::-1]

plt.figure(figsize=(10, 5))
plt.bar(range(len(feature_cols)), gb_importances[indices_gb])
plt.xticks(range(len(feature_cols)), [feature_cols[i] for i in indices_gb], rotation=45, ha="right")
plt.ylabel("Feature importance")
plt.title("Gradient boosting feature importances")
plt.tight_layout()
plt.show()
